In [ ]:
!pip install qdrant-client
!pip install sentence-transformers

In [ ]:
from qdrant_client import QdrantClient

client = QdrantClient(
    url="https://0ed5ff2c-d055-4459-aa08-8ff72d9a4f29.europe-west3-0.gcp.cloud.qdrant.io",
    api_key="UImA32LIcFUAtmobuAsSDKleHka1FG2riKCUYsU2ayK5zqGs7lYOpQ"
)

In [ ]:
client

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [48]:
payloads = [
    {
        "qa": "How to access and utilize the test reporting feature",
        "ans": "To access the test reporting feature, navigate to the Reports section in the dashboard. You can filter reports by project, test case, or test execution date. Utilize graphical representations for better insights."
    },
    {
        "qa": "How to Execute a Test",
        "ans": "To execute a test, select the test case from the test suite, choose the environment or device, and hit the 'Run' button. You can also schedule execution using the Test Scheduler."
    },
    {
        "qa": "How to Create a Test Suite",
        "ans": "Go to the Test Suites section, click on 'Create New Suite', and add test cases by selecting them from the existing list. Assign a name and description to your suite."
    },
    {
        "qa": "How to Create a Test Case",
        "ans": "Navigate to the Test Cases section, click on 'Add New', and fill out the required fields such as test name, description, steps, and expected outcomes."
    },
    {
        "qa": "How to Record a Test Case Using a Local Device",
        "ans": "Connect your device via USB, navigate to the Test Recording section, and select 'Record with Local Device'. Start interacting with the app, and the steps will be captured."
    },
    {
        "qa": "How to Record a Test Case Using a Remote Device",
        "ans": "In the Test Recording section, choose 'Remote Device'. Select a cloud service provider, and pick the device you want to record the test case on."
    },
    {
        "qa": "How Can We Add Integrations",
        "ans": "Go to the Integrations tab in settings. Click 'Add Integration', choose the desired tool (e.g., Jira, Slack, or Jenkins), and follow the setup steps."
    },
    {
        "qa": "How to Record a Test Case Using a Cloud Device",
        "ans": "Choose 'Cloud Device' under the recording options, select your preferred cloud provider, and pick a device for recording. Start interacting with the app to record the test steps."
    },
    {
        "qa": "How to Schedule Test Case Execution",
        "ans": "In the 'Test Scheduler' section, click 'Schedule New Test'. Select the test cases, choose a time and date, and specify the device or environment for execution."
    },
    {
        "qa": "How to Create a Project",
        "ans": "Navigate to the Projects section and click 'Create New Project'. Fill in the project name, description, and any relevant settings like testing environments."
    },
    {
        "qa": "How to access and utilize the test reporting features",
        "ans": "Go to the 'Reports' section, filter by project, test suite, or date, and view detailed reports including pass/fail rates, execution times, and issue tracking."
    },
    {
        "qa": "How to Create a User",
        "ans": "Go to the User Management section, click 'Add New User', enter their details, and assign a role (e.g., tester, developer, admin)."
    },
    {
        "qa": "How to Create a Team",
        "ans": "In the Team Management section, click 'Create New Team', name the team, and add users by selecting them from the available list."
    },
    {
        "qa": "What is a Test Suite?",
        "ans": "A Test Suite is a collection of test cases that are grouped together based on functionality or feature. Suites help organize tests for batch execution."
    },
    {
        "qa": "What is a Test Plan?",
        "ans": "A Test Plan outlines the scope, objectives, and resources needed for testing a project. It includes details on the test strategy, environment, and schedule."
    },
    {
        "qa": "What is a DeepLink?",
        "ans": "A DeepLink is a URL or link that directly navigates to a specific screen or function within an app. It’s useful for testing specific flows and page navigation."
    },
    {
        "qa": "How do I enable Video recording for executions?",
        "ans": "To enable video recording, go to the Execution Settings in your project, and toggle the 'Enable Video Recording' option."
    },
    {
        "qa": "What is a Page Object?",
        "ans": "A Page Object is a design pattern in test automation where each page of the application is represented by a class, encapsulating page elements and actions."
    },
    {
        "qa": "What is Test History?",
        "ans": "Test History is a log of all previous test executions, providing insights into past results, issues encountered, and changes over time."
    },
    {
        "qa": "What are the filters available on Test Cases?",
        "ans": "You can filter test cases by status (pass/fail), priority, execution history, tags, and associated project or test suite."
    },
    {
        "qa": "What methods are supported in HTTP Master?",
        "ans": "HTTP Master supports common HTTP methods such as GET, POST, PUT, DELETE, PATCH, and OPTIONS for API testing."
    },
    {
        "qa": "What is the significance/use of a Test Plan?",
        "ans": "A Test Plan ensures the testing process is well-organized, setting expectations for scope, timelines, and objectives. It acts as a reference throughout the testing cycle."
    }
]

In [49]:
vectors = model.encode([item['qa'] for item in payloads])

In [50]:
client.recreate_collection(
    collection_name="qapilot-chatbot",  # Name of your collection
    vectors_config={
        "size": 384,  # Vector size based on your model (all-MiniLM-L6-v2 outputs 384-dimensional vectors)
        "distance": "Cosine"  # You can use 'Cosine', 'Euclidean', or 'Dot' based on your retrieval approach
    }
)
/
print("Collection created!")

<ipython-input-50-9803871425f5>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Collection created!


In [51]:
client.upsert(
    collection_name="qapilot-chatbot",  # Name your collection
    points=[
        {
            "id": idx + 1,  # Unique ID for each point
            "vector": vectors[idx],  # Corresponding vector for the question
            "payload": payloads[idx]  # Store the QA pair as payload
        }
        for idx in range(len(payloads))
    ]
)
print("Data successfully added to Qdrant!")

Data successfully added to Qdrant!


In [54]:
user_query = "how to create a test plan"


query_vector = model.encode(user_query)


search_results = client.search(
    collection_name="qapilot-chatbot",
    query_vector=query_vector,
    limit=1
)
search_results

[ScoredPoint(id=15, version=0, score=0.8282936, payload={'qa': 'What is a Test Plan?', 'ans': 'A Test Plan outlines the scope, objectives, and resources needed for testing a project. It includes details on the test strategy, environment, and schedule.'}, vector=None, shard_key=None, order_value=None)]